In [ ]:
import pandas as p
url = 'https://etherscan.io/txsInternal?a=0xea3f1b409f5e9211eed087c9f5f621d1249b3e7b'
from qore import *
xp = XPath()
xcols = {
    #'to'      : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[6]/span/a/text()',
    'value'   : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[7]/text()',
}
res = xp.xpath2df(url, xcols)
p.DataFrame(res)

In [ ]:
%pylab inline
import numpy as n
from bitmex import apiRequest

%reload_ext autoreload
%autoreload 2
%pylab inline
from qoreliquid import *
import numpy as n
from bitmex import CoinMarketCap
from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
import seaborn as sns
sns.set()
cmc = CoinMarketCap()

In [ ]:
dft = cmc.getAllTokens()

In [ ]:
balance    = 1700
balanceRisk = 47.43 #float(1700) / 100 * 50
#sort = 'marketCap'
#sort = 'pcnt1h'
sort = 'balanceMarketcapPcnt'
sort = 'volumePerMarketCap'
sort = 't1Supply'
#sort = 'marketCapPcntTo1e6'

df = dft.fillna(0)
df['balanceMarketcapPcnt'] = balanceRisk / df['marketCap'] * 100

dff = df

dff['marketCapPcntTo1e6'] = dff['marketCap'] * 100 / 1e6
dff['marketCapPcntTo1e7'] = dff['marketCap'] * 100 / 1e7

dff = dff[ dff['marketCap'] >  0.0 ]
dff = dff[ dff['volume']    > 10.0 ]
dff['circulatingSupply'] = dff['marketCap'] / dff['price']
dff['volumePerMarketCap'] = dff['volume'] / dff['marketCap']

goal = 'marketCapPcntTo1e7'
dff['balanceRisk'] = balanceRisk
dff['riskOn'] = balanceRisk / dff[goal] * 100

# models
dff['t1Supply'] = dff['balanceMarketcapPcnt']**2 * dff['volumePerMarketCap']**3 / dff[goal]**2

df = dff
df = df.set_index('symbol')

df = df[df['token'] == 'ethereum']
df = df.sort_values(by=sort, ascending=False)
df#['id']

In [ ]:
for i in list(df.head(20)['id']):
    cmc.glo(i)

In [ ]:
#for i in 'bitcoin ethereum kin'.split(' '):
# b3 == pump n dump example
for i in 'b3coin'.split(' '):
    cmc.glo(i)